# Database Query Notebook

PostgreSQL 데이터베이스에서 뉴스 기사를 조회하는 노트북입니다.

In [1]:
import psycopg2
import pandas as pd
from datetime import datetime

In [2]:
# PostgreSQL 연결 설정
def get_db_connection():
    return psycopg2.connect(
        host="localhost",  # Docker service name
        database="news_db",
        user="postgres",
        password="password"
    )

In [3]:
# 접속 테스트
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='news_db',
    user='postgres',
    password='password'
)

cur = conn.cursor()
cur.execute("SELECT * FROM articles LIMIT 10;")
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
conn.close()

(1, '카카오모빌리티', 'https://www.seoultimes.news/news/article.html?no=2000058978', 'https://www.seoultimes.news/news/article.html?no=2000058978', "“사장님부터 회장님까지”...대기업 CEO '국감 증인' 줄소환", '류긍선카카오모빌리티대표(카카오택시 등 수수료 및 이용 불편), , 피터알덴우드 애플코리아 대표이사(불법 개인정보 유출과 앱마켓 독과점 지위), 김태영 웹젠 대표이사(게임 아이템 확률 조작) 등의 CEO도 마찮가지다. 이처럼 대기업 총수나 후계자, CEO들이 줄줄이 국감장 증인으로 채택되면서 이를...', '서울타임즈뉴스', 'www.seoultimes.news', datetime.datetime(2024, 9, 30, 15, 32, tzinfo=datetime.timezone.utc), '2024.10.01', datetime.datetime(2024, 11, 29, 1, 40, 57, 213465, tzinfo=datetime.timezone.utc), False, True, False)
(3, '카카오모빌리티', 'https://www.ajunews.com/view/20241001140944533', 'https://www.ajunews.com/view/20241001140944533', "올해도 기업인 줄소환… '보여주기' 전락한 국감", '이 밖에도 △장재훈 현대차 사장(대기업·중견중소기업 교란행위) △류긍선카카오모빌리티대표(카카오택시 등 수수료 및 이용 불편) △김영섭 KT 대표(한전 원격검침 인프라 구축 모뎀사업 관련) △방경만 KT&G 대표(불공정 판매 강요 문제) △강한승 쿠팡 대표(자사 우대 노출), 피터얀 반데비트 우아한형제들...', '아주경제', 'www.ajunews.com', datetime.datetime(2024, 9, 30, 15, 32, tzinfo=datetime.timezone.utc), '2024.10.0

In [ ]:
## 날짜별 기사 검색
def search_by_date(date_str):
    """
    특정 날짜의 기사를 검색합니다.
    :param date_str: 'YYYY-MM-DD' 형식의 날짜 문자열
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        id,
        title,
        publisher,
        naver_link,
        published_at
    FROM articles
    WHERE DATE(published_at) = %s
    ORDER BY published_at DESC
    """
    
    df = pd.read_sql_query(query, conn, params=[date_str])
    conn.close()
    return df

# 사용 예시:
# df = search_by_date('2024-01-15')
# display(df)

In [ ]:
## 키워드로 기사 검색
def search_by_keyword(keyword):
    """
    키워드가 포함된 기사를 검색합니다.
    :param keyword: 검색할 키워드
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        id,
        title,
        publisher,
        naver_link,
        published_at
    FROM articles
    WHERE 
        title LIKE %s OR
        main_keyword LIKE %s
    ORDER BY published_at DESC
    """
    
    search_pattern = f'%{keyword}%'
    df = pd.read_sql_query(query, conn, params=[search_pattern, search_pattern])
    conn.close()
    return df

# 사용 예시:
# df = search_by_keyword('카카오모빌리티')
# display(df)

In [ ]:
## 날짜와 키워드로 기사 검색
def search_by_date_and_keyword(date_str, keyword):
    """
    특정 날짜와 키워드로 기사를 검색합니다.
    :param date_str: 'YYYY-MM-DD' 형식의 날짜 문자열
    :param keyword: 검색할 키워드
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        id,
        title,
        publisher,
        naver_link,
        published_at
    FROM articles
    WHERE 
        DATE(published_at) = %s AND
        (title LIKE %s OR main_keyword LIKE %s)
    ORDER BY published_at DESC
    """
    
    search_pattern = f'%{keyword}%'
    df = pd.read_sql_query(query, conn, params=[date_str, search_pattern, search_pattern])
    conn.close()
    return df

# 사용 예시:
# df = search_by_date_and_keyword('2024-01-15', '카카오모빌리티')
# display(df)

In [ ]:
## 기사 내용 조회
def get_article_content(article_id):
    """
    특정 기사의 내용을 조회합니다.
    :param article_id: 기사 ID
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        a.id,
        a.title,
        a.publisher,
        c.content,
        a.published_at
    FROM articles a
    LEFT JOIN contents c ON a.id = c.article_id
    WHERE a.id = %s
    """
    
    df = pd.read_sql_query(query, conn, params=[article_id])
    conn.close()
    return df

# 사용 예시:
# df = get_article_content(3514)
# display(df)

In [ ]:
## 키워드와 날짜 범위로 댓글 검색
def search_comments_by_keyword_and_date(start_date, end_date, keyword=None):
    """
    키워드와 날짜 범위로 댓글을 검색합니다.
    
    :param start_date: 시작 날짜 'YYYY-MM-DD' 형식
    :param end_date: 종료 날짜 'YYYY-MM-DD' 형식
    :param keyword: 검색할 키워드 (선택적)
    """
    conn = get_db_connection()
    
    query = """
    SELECT 
        a.id as article_id,
        a.title as article_title,
        a.publisher,
        a.published_at as article_published_at,
        a.naver_link,
        c.id as comment_id,
        c.content as comment_content,
        c.username,
        c.timestamp as comment_timestamp,
        c.likes,
        c.dislikes
    FROM articles a
    JOIN comments c ON a.id = c.article_id
    WHERE 
        a.published_at BETWEEN %s AND %s
    """
    
    params = [start_date, end_date]
    
    if keyword:
        query += " AND (c.content LIKE %s OR a.title LIKE %s OR a.main_keyword LIKE %s)"
        search_pattern = f'%{keyword}%'
        params.extend([search_pattern, search_pattern, search_pattern])
    
    query += " ORDER BY c.timestamp DESC"
    
    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    return df

# 사용 예시:
# df = search_comments_by_keyword_and_date('2024-01-01', '2024-01-31', '카카오')
# display(df)

In [6]:
# 사용 예시:
df = search_comments_by_keyword_and_date('2024-10-01', '2024-10-31', '카카오모빌리티')
display(df)

/var/folders/m3/j7b0h30d7412rbfrmrz8rqqc0000gn/T/ipykernel_84051/3223904729.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)


,article_id,article_title,publisher,article_published_at,comment_id,comment_content,username,comment_timestamp,likes,dislikes
0,1709,"[단독]‘용산’ 퇴직한 44명 전원, 금융-공기업 등 재취업",동아일보,2024-10-06 15:32:00+00:00,1131,한동훈 억까하고 다들 한자리씩 받아갔다고 함 한심하다,choi****,2024-11-19 10:32:41+00:00,1,0
1,2739,[데스크 시각] 배민의 거짓말,서울신문,2024-10-20 15:32:00+00:00,2234,배민이나 쿠팡이나 수수료율 10%에 육박하는건 마찬가지 입니다.나라에서 수도권만이지...,blac****,2024-11-15 06:12:13+00:00,0,0
2,3899,샌프란시스코·우한 누비는 무인택시…한국 자율주행 왜 묶였나 [박용후...,중앙일보,2024-10-29 15:32:00+00:00,2526,미국과 중국은 뛰는데 국내는 플랫폼 때려잡기만 하네요.,what****,2024-11-15 04:13:47+00:00,0,0
3,3989,"롯데그룹, AI 기반의 디지털 전환 가속화 [한양경제]",경기일보,2024-10-30 15:32:00+00:00,3131,아마존 등이 15년전에 시작했던 것을~ 이제 시작해~~~? \n국내에서 꿀빨다가~ ...,sand****,2024-11-10 23:23:36+00:00,0,0
4,2657,[ZD브리핑] 이재용 삼성전자 회장 취임 2주년...국정감사 막바지,지디넷코리아,2024-10-19 15:32:00+00:00,2085,역사상 가장 무능한 개버린이 구속시켜 능력이 부족하데다 경영이 제대로 되겠니? ...,chan****,2024-11-10 11:00:08+00:00,0,0
...,...,...,...,...,...,...,...,...,...,...
2521,655,불황 넘기 바쁜데… “또 망신주기 국감 불려나가” 재계 한숨,국민일보,2024-10-01 15:32:00+00:00,709,계엄선포!국개해산!,libe****,2024-10-01 21:00:45+00:00,0,0
2522,655,불황 넘기 바쁜데… “또 망신주기 국감 불려나가” 재계 한숨,국민일보,2024-10-01 15:32:00+00:00,710,무능력한 놈들이 국민 등처 먹으면서 망신 당하고 쫒겨나야 정상 아니여! 그깟 망실이...,3lif****,2024-10-01 19:51:05+00:00,2,3
2523,655,불황 넘기 바쁜데… “또 망신주기 국감 불려나가” 재계 한숨,국민일보,2024-10-01 15:32:00+00:00,704,야당넘들 말로만국민을 위한다고 떠들면서오로지 찢 죄명방탄과 정권잡기에만 눈이 벌거...,bik0****,2024-10-01 19:41:43+00:00,11,3
2524,655,불황 넘기 바쁜데… “또 망신주기 국감 불려나가” 재계 한숨,국민일보,2024-10-01 15:32:00+00:00,705,날강도마냥 주주 삥뜯어야한다고 바쁘겠지,appl****,2024-10-01 19:29:49+00:00,5,0


In [ ]:
# 예시 실행
date_str = '2024-10-01'  # 날짜를 원하는 날짜로 변경하세요
keyword = '카카오모빌리티'

df = search_by_date_and_keyword(date_str, keyword)
display(df)